In [4]:
# Import to be able to import python package from src
import sys
sys.path.insert(0, '../../src')
import pandas as pd
import matplotlib.pyplot as plt
import ontime as on

In [5]:
ts = on.generators.random_walk().generate(start=pd.Timestamp('2022-01-01'), end=pd.Timestamp('2022-12-31'))

In [ ]:
fig, axs = plt.subplots( figsize=(20, 10))
axs.plot(ts)